In [16]:
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
import json
import csv

In [17]:
options = uc.ChromeOptions()
options.headless = False  # Hiển thị trình duyệt (False) để debug dễ hơn
options.add_argument("--disable-blink-features=AutomationControlled")
driver = uc.Chrome(version_main=134)
driver.get("https://www.starbucks.vn/store-locator")

In [18]:
soup = BeautifulSoup(driver.page_source)
results = []

for svg in soup.find_all("svg", class_="location-info"):
    try:
        data_attr = svg.get("data")
        if not data_attr:
            continue
        data = json.loads(data_attr)

        # Tách nested keys và chỉ giữ field mong muốn
        info = {
            "name": data.get("name"),
            "address": data.get("address"),
            "tel": data.get("tel"),
            "lat": data.get("location", {}).get("lat"),
            "lng": data.get("location", {}).get("lng"),
            "mon": data.get("openHour", {}).get("mon"),
            "tue": data.get("openHour", {}).get("tue"),
            "wed": data.get("openHour", {}).get("wed"),
            "thu": data.get("openHour", {}).get("thu"),
            "fri": data.get("openHour", {}).get("fri"),
            "sat": data.get("openHour", {}).get("sat"),
            "sun": data.get("openHour", {}).get("sun"),
            "services": ", ".join(data.get("services", []))
        }

        results.append(info)
    except Exception as e:
        print(f"⚠️ Lỗi khi xử lý thẻ svg: {e}")

# Xác định đúng fieldnames (chính là keys bạn extract ở trên)
fieldnames = [
    "name", "address", "tel", "lat", "lng",
    "mon", "tue", "wed", "thu", "fri", "sat", "sun", "services"
]

# Ghi ra file CSV
with open("location_info.csv", "w", encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print("✅ Đã ghi xong vào file location_info.csv")

⚠️ Lỗi khi xử lý thẻ svg: Expecting ',' delimiter: line 1 column 401 (char 400)
✅ Đã ghi xong vào file location_info.csv
